In [1]:
from __future__ import print_function
#token:
#http://127.0.0.1:8888/?token=213d7c96a44367f5295370d6941718325f47032e4a519a21

import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np

# Import biomag data
biomag_labeled_1 = sio.loadmat(r"D:\python_project\wip\baseline\data_labeled_for_py_baseline1_1.mat")
#"lib": [r"phd\orai\6\History-From-the-Quaran-to-Bin-Laden_txt.txt",]

In [2]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 128                         # -s         -o 128
display_step = 100
tf.set_random_seed(1234)

# Network Parameters
n_hidden_1 = 500 # 1st layer number of neurons
n_hidden_2 = 500 # 2nd layer number of neurons
num_input = 254 # biomag data input (shape: *)
num_classes = 12 # biomag total classes (1-12)

#Data preprocessing -s
biomag_labeled_1['y_train'] = biomag_labeled_1['y_train'].T

biomag_labeled_1['y_train'] = np.reshape(biomag_labeled_1['y_train'],2094)

print('Tanítóhalmaz címkéinek formája: ',biomag_labeled_1['y_train'].shape)

#y labels -1
biomag_labeled_1['y_train'] = biomag_labeled_1['y_train']-1

Tanítóhalmaz címkéinek formája:  (2094,)


In [3]:
#int(biomag_labeled_1['y_train'][:])
biomag_labeled_1['y_train']

array([1., 0., 1., ..., 1., 1., 7.])

In [5]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'train_data': biomag_labeled_1['x_train']}, y=biomag_labeled_1['y_train'],
    batch_size=batch_size, num_epochs=None, shuffle=True)

# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['train_data']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1, activation= tf.nn.relu)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2, activation= tf.nn.relu)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

In [6]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    # Build the neural network
    logits = neural_net(features)

    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits (              #rank mismatch -s   softmax_cross_entropy_with_logits
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())

    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [7]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\aqwis\\AppData\\Local\\Temp\\tmpzjjpm0ug', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000027EED904550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Running local_init_op.


...

INFO:tensorflow:loss = 0.41178990047368647, step = 800 (0.291 sec)


INFO:tensorflow:global_step/sec: 327.651


INFO:tensorflow:loss = 0.24531774834118036, step = 900 (0.305 sec)


INFO:tensorflow:Saving checkpoints for 1000 into C:\Users\aqwis\AppData\Local\Temp\tmpzjjpm0ug\model.ckpt.


INFO:tensorflow:Loss for final step: 0.3768066034131845.


In [9]:
# Evaluate the Model

#process test data
biomag_labeled_1['y_test'] = np.reshape(biomag_labeled_1['y_test'],206)
biomag_labeled_1['y_test'] = biomag_labeled_1['y_test']-1
# Define the input function for evaluating
input_fn_test = tf.estimator.inputs.numpy_input_fn(
    x={'train_data': biomag_labeled_1['x_test']}, y=biomag_labeled_1['y_test'],
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn_test)


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2018-05-08-12:35:08

INFO:tensorflow:Graph was finalized.

INFO:tensorflow:Restoring parameters from C:\Users\aqwis\AppData\Local\Temp\tmpzjjpm0ug\model.ckpt-1000

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.

INFO:tensorflow:Finished evaluation at 2018-05-08-12:35:09


INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.8737864, global_step = 1000, loss = 0.36327505


{'accuracy': 0.8737864, 'global_step': 1000, 'loss': 0.36327505}

In [20]:
biomag_labeled_1['y_train'].shape

biomag_labeled_1['y_train'] = biomag_labeled_1['y_train'].T

biomag_labeled_1['y_train'] = np.reshape(biomag_labeled_1['y_train'],2094)

In [10]:
biomag_labeled_1['y_test'] = np.reshape(biomag_labeled_1['y_test'],206)

In [11]:
biomag_labeled_1['y_test'].shape

(206,)

In [3]:
!jupyter nbconvert --to script baseline_ann_2.ipynb

This application is used to convert notebook files (*.ipynb) to various other
formats.


Options

-------



Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.


--debug

    set log level to logging.DEBUG (maximize logging output)

--generate-config

    generate default config file

-y

    Answer yes to any questions instead of prompting.

--execute

    Execute the notebook prior to export.

--allow-errors

    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.

--stdin

    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'

--stdout

    Write notebook output to stdout instead of files.

--inplace

    Run nbconvert in 

[NbConvertApp] WARNING | pattern 'baseline_ann_2.ipynb' matched no files
